# LangChain と Zero-Shot プロンプトを使用して Bedrock モデルを呼び出す

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 導入

このノートブックでは、サポートエンジニアから受けたカスタマーサービスの質について否定的なフィードバックを寄せた顧客へのメール返信を、LLM を使用して生成する方法を示します。 

この例では、Bedrock が提供する Anthropic Claude モデルを使用します。ここでは [LangChain](https://python.langchain.com/docs/get_started/introduction.html) と統合されている Bedrock バージョンを使用します。LangChain は言語モデルを利用したアプリケーションを開発するためのフレームワークです。このフレームワークの主な特徴により、さまざまなコンポーネントを連結して高度なユースケースを作成することで、大規模言語モデルを拡張できます。

このノートブックでは、LangChain が提供する Bedrock API を使用します。この例で使用されているプロンプトは、プロンプト以外に追加のコンテキストを提供していないため、Zero-Shot プロンプトと呼ばれています。

**Note:** *このノートブックは AWS 内外問わず実行することができます*

#### コンテキスト
先ほどの `00_generate_w_bedrock.ipynb` の例では、Boto3 クライアントを使用して Amazon Bedrock API と通信する方法について説明しました。このノートブックでは、LangChain フレームワークを同様のユースケースに活用するために、もう少し複雑さを加えようとします。LangChain フレームワーク内での Amazon Bedrock インテグレーションの使用方法と、`PromptTemplate` を使用してテキストを生成する方法を探ります。

#### パターン
Amazon Bedrock API の LangChain 実装に、タスク、命令、入力を提供するだけで、追加の例は示さずに出力を生成できます。ここでの目的は、強力な LLM が目の前のタスクをいかに簡単に理解し、説得力のある出力を生成するかを示すことです。

![](./images/bedrock_langchain.jpg)

#### ユースケース
Amazon Bedrock のモデルの生成機能を実証するために、E メール生成のユースケースを見てみましょう。

#### ペルソナ
あなたは AnyCompany のカスタマーサービスマネージャーである Bob ですが、一部の顧客はカスタマーサービスに満足しておらず、カスタマーサポートエンジニアが提供するサービスに否定的なフィードバックをしています。今度は、サービスの質が悪かったことを謙虚に謝罪する顧客に応え、信頼を取り戻したいと考えています。以前のメールでのやり取りから得た顧客の感情に合わせて、ヒューマンフレンドリーなメールを大量に生成するには、LLM の助けが必要です。

#### 実装
このユースケースを実現するために、このノートブックでは、顧客の以前のメールに基づいてサンキューノート付きのメールを生成する方法を示します。ここでは Amazon Bedrock LangChain インテグレーションを使用する Amazon Titan Text Large モデルを使用します。 

## セットアップ

このノートブックの以降を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrockに接続する必要があります。

セットアップの仕組みと ⚠️ **変更の要否** の詳細については [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) を参照ください。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## LangChain インテグレーションを使用して Bedrock クライアントを起動します

llms から Bedrock クラスのインスタンスを作成することから始めましょう。これは Amazon Bedrock で利用できるモデルの `model_id` を想定しています。 

オプションで、以前に作成した boto3 の `client` のほか、`temperature`、`topP`、`maxTokenCount`、`stopSequences` などのパラメータを保持できる `model_kwargs` を渡すことができます (パラメータの詳細については Amazon Bedrock コンソールで確認できます)。

Amazon Bedrock で利用可能なテキスト生成モデルには、次の ID があります。

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v2`
- `anthropic.claude-v2`

`model_kwargs` はモデルが異なればサポートも異なることに注意してください。

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

`client` を LangChain に渡すことで、ライブラリが以前に設定を確認したのと同じ boto3 クライアントを使用していることを確認できるはずです。

In [ ]:
print(boto3_bedrock)
print(textgen_llm.client)

LangChain は Amazon Bedrock API を抽象化し、ユースケースを簡単に構築できるようにしました。プロンプトを渡すと、自動的に適切な API にルーティングされて応答が生成されます。テキスト出力をそのまま取得するだけで、応答本文から結果を抽出する必要はありません。

カスタマーサービスマネージャーが顧客に送信するメールを生成するプロンプトを準備しましょう。

In [ ]:
response = textgen_llm("""

Human: Write an email from Bob, Customer Service Manager, 
to the customer "John Doe" that provided negative feedback on the service 
provided by our customer support engineer.

Assistant:""")

print_ww(response)

## 結論
これで、Amazon Bedrock API に抽象化レイヤーを提供する `LangChain` フレームワークを試してみました。このフレームワークを使用して、顧客からの否定的なフィードバックに応答する E メールを生成するユースケースを見てきました。

### 発展要素
- Anthropic Claude や AI21 Labs の Jurassic モデルなど、Amazon Bedrock から入手できるさまざまなモデルを試すために、このノートブックを改造する
- プロンプトを特定のユースケースに変更し、さまざまなモデルの出力を評価する
- トークンの長さを複数変更して試すことで、サービスのレイテンシーと応答性を理解する
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を得る

## ありがとうございました